In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv('./datasets/train.csv')

sample_df = pd.read_csv('./datasets/sample_submission.csv') 

In [3]:
train_df = train_df.dropna()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404287 entries, 0 to 404289
Data columns (total 6 columns):
id              404287 non-null int64
qid1            404287 non-null int64
qid2            404287 non-null int64
question1       404287 non-null object
question2       404287 non-null object
is_duplicate    404287 non-null int64
dtypes: int64(4), object(2)
memory usage: 21.6+ MB


In [4]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
train_df['text'] = train_df['question1'] + train_df['question2']

In [6]:
stops_word = stopwords.words("english")
tfidf_vec = TfidfVectorizer(analyzer='word',  max_features=1000, stop_words=stops_word)

In [7]:
X_train = tfidf_vec.fit_transform(train_df['text'])

In [8]:
X_train.shape

(404287, 1000)

In [9]:
Y_train = train_df['is_duplicate']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train.toarray(), Y_train, test_size = 0.1, 
                                                   stratify=Y_train, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# rf_clf = RandomForestClassifier()
# rf_clf.fit(X_train, Y_train)

In [13]:
lin_log = LogisticRegression()
lin_log.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
Y_pred = lin_log.predict(X_test)
loss = np.sqrt(mean_squared_error(Y_pred, Y_test))

In [15]:
loss

0.5449903845763835

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_pred, Y_test)

0.7029854807192857

In [39]:
# test_df
test_df = pd.read_csv('./datasets/test.csv')

In [40]:

# test_df = test_df.dropna()
test_df['text'] = test_df['question1'] + test_df['question2']

In [41]:
test_df['text'] = test_df['text'].fillna("")
# 

In [42]:
X_test_f = tfidf_vec.transform(test_df['text']).toarray()

In [43]:
Y_test_f = lin_log.predict(X_test_f)

In [45]:
sample_df.head()

,test_id,is_duplicate
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [51]:
result = pd.DataFrame({
    'test_id':sample_df['test_id'],
    'is_duplicate': Y_test_f
})
result.to_csv("./result.csv", index=False)